In [12]:
from selenium import webdriver
import numpy as np
import pandas as pd
import re
import sys
import time
from credentials import EMAIL, PASSWORD

In [14]:
cities = ["2014-new-york-city","2015-boston","2014-san-francisco",
      "2014-chicago","2014-honolulu","2015-walt-disney-world","2015-los-angeles",
      "2015-san-diego-rock-n-roll","2014-twin-cities","2014-berlin","2014-dublin",
     "2015-big-sur","2015-london","2015-bay-of-mont-saint-michel","2014-savannah-rock-n-roll",
         "2014-auckland","2014-amsterdam"]
run_type = "marathon"
link = "https://www.strava.com/running-races/"
races=[]
for city in cities:
    races.append(link + city + "-" + run_type)

In [15]:
browser = webdriver.Chrome()
browser.get('https://app.strava.com/login')
browser.find_element_by_name('email').send_keys(EMAIL)
browser.find_element_by_name('password').clear()
browser.find_element_by_name('password').send_keys(PASSWORD)
browser.find_element_by_id('login-button').submit()
time.sleep(2)

In [16]:
for r in xrange(len(cities)):    
    browser.get(races[r]) #load page
    html_code = browser.page_source.encode('utf-8')
    num_pages = 0
    num = re.search(r'(\d+)<\/a> <a class=\"next_page\"',html_code)
    if num: num_pages = int(num.group(1))
    html_file = open("../raw_html/"+cities[r]+".html",'w')
    html_file.write(html_code)
    time.sleep(3)
    for i in xrange(num_pages-1):
        time.sleep(1)
        error = False
        while not error:
            try:
                browser.find_element_by_class_name("next_page").click()
                error = True
            except:
                pass
        time.sleep(1)
        html_code = browser.page_source.encode('utf-8')
        html_file.write(html_code) 
    button = browser.find_element_by_xpath('//*[@id="weekly-training-controls"]/div[3]/div')
    for i in xrange(1,8):
        error2 = False
        while not error2:
            try:
                button.click()
                error2 = True
                time.sleep(2)
                browser.find_element_by_xpath('//*[@id="weekly-training-controls"]/div[3]/ul/li['+str(i)+']').click()
                time.sleep(2)  
                html_code = browser.page_source.encode('utf-8')
                html_file.write(html_code)
            except:
                pass
    html_file.close        
browser.quit()    